In [1]:
BATCH_SIZE = 16
EDGE_CROP = 16
GAUSSIAN_NOISE = 0.1
UPSAMPLE_MODE = 'SIMPLE'
# downsampling inside the network
NET_SCALING = None
# downsampling in preprocessing
IMG_SCALING_2 = (2,2)
IMG_SCALING_3 = (3,3)
# number of validation images to use
VALID_IMG_COUNT = 900
# maximum number of steps_per_epoch in training
MAX_TRAIN_STEPS = 200000//BATCH_SIZE
MAX_TRAIN_EPOCHS = 5
SHUFFLE_BUFFER = 100000

In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
from skimage.io import imread
from sklearn.model_selection import train_test_split
from keras import backend as K
from tensorflow.python.keras.utils import multi_gpu_model

from tensorflow.python import keras as keras
from tensorflow.python.keras.models import load_model
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.losses import binary_crossentropy
from models.resnet50_classif import get_resnet50_classif
from models.resnet50_classif_tf import get_resnet50_classif_tf
from tensorflow.python.keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau, TensorBoard, Callback

#import tensorflow as tf
#import os


Using TensorFlow backend.


In [3]:
import tensorflow as tf
with tf.Session() as sess:
    devices = sess.list_devices()
for device in devices:
    print(device)
    
if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

_DeviceAttributes(/job:localhost/replica:0/task:0/device:CPU:0, CPU, 268435456, 13309358446037689541)
_DeviceAttributes(/job:localhost/replica:0/task:0/device:XLA_GPU:0, XLA_GPU, 17179869184, 2075061398288275319)
_DeviceAttributes(/job:localhost/replica:0/task:0/device:XLA_GPU:1, XLA_GPU, 17179869184, 17715226185103306863)
_DeviceAttributes(/job:localhost/replica:0/task:0/device:XLA_GPU:2, XLA_GPU, 17179869184, 12222746051127887753)
_DeviceAttributes(/job:localhost/replica:0/task:0/device:XLA_GPU:3, XLA_GPU, 17179869184, 9579791254581600488)
_DeviceAttributes(/job:localhost/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 13617999991671828380)
_DeviceAttributes(/job:localhost/replica:0/task:0/device:GPU:0, GPU, 15560255079, 1758207236893325804)
_DeviceAttributes(/job:localhost/replica:0/task:0/device:GPU:1, GPU, 15560192820, 7597960524765833202)
_DeviceAttributes(/job:localhost/replica:0/task:0/device:GPU:2, GPU, 15560753152, 605787587743216366)
_DeviceAttributes(/job:localhost

In [4]:
ship_dir = '../../data/airbus_ship_detection/'
train_image_dir = os.path.join(ship_dir, 'train')# Images for training
test_image_dir = os.path.join(ship_dir, 'test')# Images for testing
label_dir = os.path.join(ship_dir, 'train_ship_segmentations_v2.csv')# Images for testing
masks = pd.read_csv(label_dir, engine="python") # Markers for ships

In [5]:
def _parse_function(proto):
    # define your tfrecord again. Remember that you saved your image as a string.
    keys_to_features = {'image_raw': tf.FixedLenFeature([], tf.string),
                        "label": tf.FixedLenFeature([], tf.int64)}
    
    # Load one example
    parsed_features = tf.parse_single_example(proto, keys_to_features)
    
    # If image compressed
    image = tf.image.decode_image(parsed_features['image_raw'], dtype=tf.dtypes.float32)
    # else raw list
    #image = tf.decode_raw(parsed_features['image_raw'], tf.float32)
    
    return image, parsed_features["label"]

In [6]:
def create_dataset(filepath, batch_size=BATCH_SIZE):
    
    # This works with arrays as well
    dataset = tf.data.TFRecordDataset(filepath)
    # Maps the parser on every filepath in the array. You can set the number of parallel loaders here
    dataset = dataset.map(_parse_function, num_parallel_calls=8)
    
    # This dataset will go on forever
    dataset = dataset.repeat()
    
    # Set the number of datapoints you want to load and shuffle 
    dataset = dataset.shuffle(256)
    
    # Set the batchsize
    dataset = dataset.batch(batch_size)
    
    # Create an iterator
    iterator = dataset.make_one_shot_iterator()
    
    # Create your tf representation of the iterator
    image, label = iterator.get_next()
    
    # Bring your picture back in shape
    image = tf.reshape(image, [-1, 768, 768, 3])
    
    # Create a one hot array for your labels
    label = tf.one_hot(label, 1)
    
    return image, label

In [7]:
os.path.join(ship_dir, "training_clf.tfrecords")

'../../data/airbus_ship_detection/training_clf.tfrecords'

In [8]:
tf_records_path_train = os.path.join(ship_dir, "training_clf.tfrecords")
tf_records_path_val = os.path.join(ship_dir, "validation_clf.tfrecords")

In [9]:
image, label = create_dataset(tf_records_path_train, batch_size=BATCH_SIZE)

Instructions for updating:
Colocations handled automatically by placer.


In [10]:
weights_path_clf_scal2 = "weights_models/{}_weights.best.hdf5".format('clf_Scal2_tf_learn')

In [11]:
model_clf_scal2 = load_model(weights_path_clf_scal2)
model_clf_scal2.summary()

Instructions for updating:
`normal` is a deprecated alias for `truncated_normal`
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, None, None, 6 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalizationV1) (None, None, None, 6 256         conv1[0][0]                      
____________________________

In [12]:
from tensorflow.python.keras import layers, models

In [13]:
Inputs = layers.Input(tensor=image)
x = model_clf_scal2 (Inputs)
x = layers.Flatten() (x)
x = layers.Dense(128, activation='relu') (x)
Outputs = layers.Dense(1, activation='sigmoid') (x)

model =  models.Model(inputs=[Inputs], outputs=[Outputs])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 768, 768, 3)       0         
_________________________________________________________________
resnet50 (Model)             multiple                  23587712  
_________________________________________________________________
flatten (Flatten)            (None, 1179648)           0         
_________________________________________________________________
dense (Dense)                (None, 128)               150995072 
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 174,582,913
Trainable params: 174,529,793
Non-trainable params: 53,120
_________________________________________________________________


In [14]:
model.get_layer('resnet50').summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, None, None, 6 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalizationV1) (None, None, None, 6 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [15]:
from tensorflow.python.keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau, TensorBoard, Callback
weight_path = "weights_models/{0}_weights.hdf5".format('model_1_scal1')

checkpoint = ModelCheckpoint(weight_path, monitor='val_loss', verbose=0, save_best_only=True, mode='min', save_weights_only=True)

reduceLROnPlat = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                                   patience=2, verbose=1, mode='min',
                                   min_delta=0.001, cooldown=1, min_lr=1e-7)

early = EarlyStopping(monitor="val_loss", mode="min", verbose=2,
                      patience=2) # probably needs to be more patient, but kaggle time is limited

tensorboard =  TensorBoard(log_dir="../logs/log1", update_freq='batch')

callbacks_list = [checkpoint, reduceLROnPlat, early]

In [16]:
#run_opts = tf.RunOptions(report_tensor_allocations_upon_oom = True)

parallel_model = multi_gpu_model(model, gpus=4)
parallel_model.compile(optimizer=Adam(), loss=binary_crossentropy, 
              metrics=["accuracy"],
              target_tensors=[label])
# ,options = run_opts)

In [17]:
loss_history = [parallel_model.fit(epochs=MAX_TRAIN_EPOCHS,
                                   steps_per_epoch=MAX_TRAIN_STEPS,
                                 callbacks=callbacks_list)]

Instructions for updating:
Use tf.cast instead.
Epoch 1/5
  452/12500 [>.............................] - ETA: 2:14:44 - loss: 1.7194 - acc: 0.7713

KeyboardInterrupt: 